## Read me

Je n'ai encore jamais tenté de faire tourner le script sur la totalité de lightnovelworld.com (j'étais allée assez loin dans la collecte sans rencontrer d'erreur mais il est possible qu'il en traîne encore).

Je n'ai pas encore réussi à inclure les langues d'origine (principalement chinois, japonais et coréen) des livres dans la collecte de données. Je me pencherai à nouveau dessus quand j'en aurai le temps et actualiserai le fichier si j'y parviens.

Il est nécessaire que l'appareil qui lance le script ait installé Chrome.

Le nombre de pages présentant les livres du site (avant-dernière cellule, ligne 2) a été rentré manuellement, il était de 86 la dernière fois que j'y suis allée (fin juin 2024). Il pourrait être à actualiser s'il venait à changer.

En raison des coupures de réseau, j'ai dû ajouter des conditions afin que la collecte ne stoppe pas. L'avantage est qu'à présent elle saute les livres qui n'ont pas pu être collectés (et les précise à la fin de la collecte) ; l'inconvénient est que le script met du temps à s'arrêter si on veut l'interrompre...

In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from datetime import datetime

In [ ]:
# Initialisation du fichier CSV
def initialize_csv(file_name):
    with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow([
            'Novel Title', 'Alternative Title', 'Author Name', 'Rank', 'Rating', 
            'Chapters', 'Views', 'Bookmarked', 'Status', 'Categories', 'Summary', 
            'Tags', 'Last Updated', 'First Chapter Release'
        ])

# Ecriture des données dans le fichier CSV
def write_to_csv(file_name, data):
    with open(file_name, mode='a', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(data)

In [ ]:
# Collecte des liens pour une page du site
def get_book_links(url):
    driver = webdriver.Chrome()
    driver.get(url)

    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')

    h4_tags = soup.find_all('h4', class_='novel-title text2row')

    links = []

    for h4 in h4_tags:
        a_tag = h4.find('a')
        if a_tag and 'href' in a_tag.attrs:
            links.append(a_tag['href'])

    driver.quit()
    
    return links

In [ ]:
# Collecte des informations (sans les dates) sur un livre
def get_book_info(url):
    driver = webdriver.Chrome()
    driver.get(url)

    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')

    novel_info = soup.find('div', class_='novel-info')

    if novel_info:

        title_tag = novel_info.find('h1', class_='novel-title text2row')
        title = title_tag.text.strip() if title_tag else 'N/A'

        alt_title_tag = novel_info.find('h2', class_='alternative-title text1row')
        alt_title = alt_title_tag.text.strip() if alt_title_tag else 'N/A'

        author_tag = novel_info.find('a', class_='property-item')
        author = author_tag.text.strip() if author_tag else 'N/A'

        rank_tag = novel_info.find('div', class_='rank')
        rank = rank_tag.find('strong').text.strip().replace('RANK ', '') if rank_tag and rank_tag.find('strong') else 'N/A'

        rating_tag = novel_info.find('div', class_='rating-star')
        rating = rating_tag.find('strong').text.strip() if rating_tag and rating_tag.find('strong') else 'N/A'

        chapters_tag = novel_info.find('div', class_='header-stats')
        chapters = chapters_tag.find('strong').text.strip().replace('<i class="icon-book-open">::before</i>','') if chapters_tag and chapters_tag.find('strong') else 'N/A'

        views_tag = novel_info.find('i', class_='icon-book-open')
        views = views_tag.find_next('strong').text.strip().replace('"', '') if views_tag and views_tag.find_next('strong') else 'N/A'

        bookmarks_tag = novel_info.find('i', class_='icon-eye')
        bookmarks = bookmarks_tag.find_next('strong').text.strip().replace('"', '') if bookmarks_tag and bookmarks_tag.find_next('strong') else 'N/A'

        status_tag = novel_info.find('i', class_='icon-bookmark')
        status = status_tag.find_next('strong').text.strip().replace('"', '') if status_tag and status_tag.find_next('strong') else 'N/A'

        categories_list = []
        categories_tag = novel_info.find('div', class_='categories')
        if categories_tag:
            category_links = categories_tag.find_all('a', class_='property-item')
            for link in category_links:
                categories_list.append(link.text.strip())
        categories = ', '.join(categories_list) if categories_list else 'N/A'

        summary_tag = soup.find('div', class_='summary')
        summary = summary_tag.find('p').text.strip().replace('"', '') if summary_tag and summary_tag.find('p') else 'N/A'

        tags_list = []
        tags_section = soup.find('div', class_='tags')
        if tags_section:
            tag_links = tags_section.find_all('a')
            for link in tag_links:
                tag_text = link.text.strip()
                if tag_text != 'Show More':
                    tags_list.append(tag_text)
        tags = ', '.join(tags_list) if tags_list else 'N/A'
        
        driver.quit()

        return {
            'Novel Title': title,
            'Alternative Title': alt_title,
            'Author Name': author,
            'Rank': rank,
            'Rating': rating,
            'Chapters': chapters,
            'Views': views,
            'Bookmarked': bookmarks,
            'Status': status,
            'Categories': categories,
            'Summary': summary,
            'Tags': tags
        }

In [ ]:
# Collecte des dates pour un livre
def get_book_dates(url):
    driver = webdriver.Chrome()
    driver.get(url)

    page_content = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(page_content, 'html.parser')

    novel_info = soup.find('article', id='chapter-list-page')

    updated = 'N/A'
    release = 'N/A'

    if novel_info:

        updated_tag = novel_info.find('time', {'datetime': True})
        if updated_tag:
            updated = updated_tag['datetime']
        
        chapter_list_section = novel_info.find('section', id='chpagedlist')
        
        if chapter_list_section:
            release_tag = chapter_list_section.find('time', {'datetime': True})
            if release_tag:
                release = release_tag['datetime']

    return {
        'Last Updated': updated,
        'First Chapter Release': release
    }

In [ ]:
# Nom du fichier CSV
base_filename = 'lightnovelworld'
current_date = datetime.now().strftime('%Y-%m-%d')
csv_file = f"{base_filename}_{current_date}.csv"

# Initialisation du fichier CSV
initialize_csv(csv_file)

In [ ]:
url_base = 'https://www.lightnovelworld.com/browse/genre-all-25060123/order-popular/status-all'
total_pages = 86 # A mettre à jour
liant = '?page='

# Liste pour stocker les numéros de passage des œuvres manquantes
failed_attempts = []

# Boucle pour itérer sur les numéros de page
for page_num in range(1, total_pages + 1):
    page_url = f"{url_base}{liant}{page_num}"
    book_links = get_book_links(page_url)
    
    for index, link in enumerate(book_links, start=1):
        try:
            true_link = f"https://www.lightnovelworld.com{link}"
            chapters_link = f"https://www.lightnovelworld.com{link}/chapters"
            info = get_book_info(true_link)
            dates = get_book_dates(chapters_link)

            # Combiner les informations des livres et les dates
            book_data = [
                info['Novel Title'], info['Alternative Title'], info['Author Name'], 
                info['Rank'], info['Rating'], info['Chapters'], info['Views'], 
                info['Bookmarked'], info['Status'], info['Categories'], 
                info['Summary'], info['Tags'], dates['Last Updated'], 
                dates['First Chapter Release']
            ]
            
            # Écrire les données dans le fichier CSV
            write_to_csv(csv_file, book_data)
            print(info)
        except Exception as e:
            print(f"An error occurred with {true_link}: {e}")
            failed_attempts.append((page_num, index))
            continue

In [ ]:
# Afficher les numéros de passage des œuvres manquantes
if failed_attempts:
    print("\nList of failed attempts (Page number, Link index):")
    for attempt in failed_attempts:
        print(attempt)

print('Data collect complete!')